In [1]:
#计算test的预测准确率
#基于用户的协同过滤算法
import os
import numpy as np
#import matplotlib.pyplot as plt
#import seaborn as sbn
import pandas as pd
#正常显示中文标签和负号
#plt.rcParams['font.sans-serif'] = ['SimHei']  #用来正常显示中文标签
#plt.rcParams['axes.unicode_minus'] = False  #用来正常显示负号
import gc

In [2]:
DATA_PATH = os.path.join(os.getcwd(),'data','input')

user_similarity = np.load(os.path.join(DATA_PATH,'user_similarity.npy'))


In [3]:
TRAIN = os.path.join(DATA_PATH,'train.csv')
train = pd.read_csv(TRAIN)
train = train[['msno','song_id','target']]

In [4]:
n_users = 21965
n_items = 30113
train_data_matrix = np.zeros((n_users,n_items))
for line in train.itertuples():
    train_data_matrix[line[1]-1,line[2]-1] = line[3]

In [5]:
#下一步是预测，已经创建了相似性矩阵
#下面基于相似性举证做预测
#基于用户的评分可以根据用户均值评分
#基于产品评分缺乏针对性
def predict(ratings,similarity,type = 'user'):
    if type == 'user':
        mean_user_rating = ratings.mean(axis = 1)
        #You use np.newaxis so that mean_user_rating has same format as ratings
        ratings_diff = (ratings - mean_user_rating[:,np.newaxis])
        pred = mean_user_rating[:,np.newaxis] + similarity.dot(ratings_diff)/np.array([np.abs(similarity).sum(axis=1)]).T
    elif type == 'item':
        pred = ratings.dot(similarity)/np.array([np.abs(similarity).sum(axis=1)])     
    return pred

In [6]:
test_user_prediction = predict(train_data_matrix,user_similarity,type = 'user')

In [7]:
#print(type(test_user_prediction))


In [9]:
del user_similarity
gc.collect()

15

In [10]:
TEST = os.path.join(DATA_PATH,'test.csv')
test = pd.read_csv(TEST)
test = test[['msno','song_id','target']]
test['user_pre_target'] =0
print(len(test))
test.head()


1216963


,msno,song_id,target,user_pre_target
0,6051,23194,1,0
1,17230,6575,1,0
2,4577,6861,1,0
3,21721,29949,0,0
4,17563,29373,0,0


In [18]:
#test.iat[1,1]

6575

In [11]:
#赋值
for i in range(0,1216963):
    test.iat[i,3] = test_user_prediction[test.iat[i,0],test.iat[i,1]]

In [13]:
test.head()


,msno,song_id,target,user_pre_target
0,6051,23194,1,0
1,17230,6575,1,0
2,4577,6861,1,0
3,21721,29949,0,0
4,17563,29373,0,0


In [12]:
#模型评价
from sklearn import metrics
from scipy import interp
cm_test = metrics.confusion_matrix(test['target'],test['user_pre_target'])
print(cm_test)

[[568010      0]
 [648953      0]]
